In [ ]:
%config IPCompleter.greedy=True

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

In [2]:
import import_ipynb
from functions import get_detailed_aggregate_data
from functions import get_raw_data
from functions import get_aggregate_functions
from visuals import colors
from visuals import map_position_to_color

importing Jupyter notebook from functions.ipynb
importing Jupyter notebook from visuals.ipynb


In [3]:
import os
import requests
import pandas

# Get parameters
# SEASON = os.environ["FPL_SEASON"]
SEASON = '2018-19'

# Get current gameweek
# URL = "https://fantasy.premierleague.com/api/bootstrap-static"
# DATA = requests.get(URL).json()
# CURR_GW = DATA['next-event'] - 1
CURR_GW = 38

BASE_PATH = 'C:/Users/aelek/source/antoniaelek/fantasy-premier-league/'
BASE_PATH = '../'

In [81]:
agg_data = get_detailed_aggregate_data(BASE_PATH, SEASON)
raw_data = get_raw_data(BASE_PATH, SEASON)
df = pandas.merge(agg_data, raw_data, on='name', how='outer')

In [96]:
goalkeepers = df[df.position == 'Goalkeeper']
defenders = df[df.position == 'Defender']
midfielders = df[df.position == 'Midfielder']
forwards = df[df.position == 'Forward']

In [97]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go


def get_agg_features(features, aggregates):
    agg_features = []
    for feature in features:
        for agg in aggregates:
            agg_features.append(agg + '_' + feature)
    return agg_features


def get_trace(df, x_metrics, y_metrics, color):
    return go.Scatter(
        x = df[x_metrics],        
        y = df[y_metrics],
        text = df['name'],        
        mode = 'markers',        
        marker=dict(color=color, size=20),
        hovertemplate = "<b>%{text}</b><br><br>" +
            y_metrics+": %{y:.2f}</br>"+
            x_metrics+": %{x:.2f}</br>"+
            "<extra></extra>")


def get_data(df, x_metrics, y_metrics, color):
    data = []
    for x in x_metrics:
        for y in y_metrics:
            data.append(get_trace(df, x, y, color))
    
    for el in data[1:]:
        el['visible'] = 'legendonly'
    
    return data


def get_layout(df, x_metrics, y_metrics, title):  
    buttons=[]
    i = 0
    for x in x_metrics:
        for y in y_metrics:
            template = [False] * len(y_metrics)
            template[i] = True
            buttons.append(dict(label = y, method = 'update', args = [{'visible': template}]))        
            i+=1
        
    updatemenus = list([
        dict(active=0,
             pad = {'r': 0, 't': 10},
             x = 0,
             y = 1.18,
             xanchor = 'left',
             buttons=buttons)])

    layout = go.Layout(
        hovermode = 'closest',
        showlegend=False,
        updatemenus=updatemenus, 
        title=go.layout.Title(
            text=title,
                font=dict(
                    size=21
                )
        ),
        xaxis=go.layout.XAxis(
            title=go.layout.xaxis.Title(
                text=x,
                font=dict(
                    size=18
                )
            )
        ),
        yaxis=go.layout.YAxis(
            title=go.layout.yaxis.Title(
                text='',
                font=dict(
                    size=18
                )
            )
        )
    )
    return layout


def get_figure(df, x_metric, y_metrics, title = '', color='rgba(101,255,71, 0.4)'):    
    data = get_data(df, [x_metric], y_metrics, color)
    layout = get_layout(df, [x_metric], y_metrics, title)
    
    return go.Figure(data=data, layout=layout)

In [98]:
generic_metrics = ['total_points', 'bonus', 'bps', 'cost', 
                   'loaned_in', 'loaned_out','transfers_balance', 'transfers_in', 'transfers_out']

# Goalkeepers

In [99]:
game_metrics_gkp = ['clean_sheets', 'saves', 'goals_conceded', 'penalties_saved', 'penalties_conceded',
                    'key_passes', 'own_goals', 'recoveries', 'tackled', 'tackles', 'clearances_blocks_interceptions',
                    'errors_leading_to_goal', 'errors_leading_to_goal_attempt', 
                    'fouls', 'red_cards', 'yellow_cards']

## Performance

In [105]:
gk1 = get_figure(goalkeepers[(goalkeepers['sum_total_points']>0)], 
                 'sum_total_points', 
                 ['sum_clean_sheets','sum_minutes','sum_penalties_saved', 'sum_bonus'],
                 '',
                 map_position_to_color('Goalkeeper'))
plotly.offline.iplot(gk1)

## Hero or zero?

In [106]:
errors = goalkeepers['sum_errors_leading_to_goal'] 
+ goalkeepers['sum_errors_leading_to_goal_attempt']
+ goalkeepers['sum_penalties_conceded']
+ goalkeepers['sum_own_goals']
+ goalkeepers['sum_fouls']
+ goalkeepers['sum_red_cards']
+ goalkeepers['sum_yellow_cards']
tmp_gkp = goalkeepers.assign(errors=errors)
gk2 = get_figure(tmp_gkp[(tmp_gkp['sum_saves']>0)], 
                 'errors', 
                 ['sum_saves','sum_penalties_saved'],
                 '',
                 map_position_to_color('Goalkeeper'))
plotly.offline.iplot(gk2)

# Defenders

In [103]:
game_metrics_def = ['clean_sheets', 'goals_conceded', 'penalties_conceded', 'own_goals','goals_scored', 'assists',
                    'big_chances_missed', 'penalties_missed'
                    'key_passes', 'recoveries', 'tackles', 'clearances_blocks_interceptions','open_play_crosses', 
                    'errors_leading_to_goal', 'errors_leading_to_goal_attempt', 
                    'fouls', 'red_cards', 'yellow_cards']

## Performance

In [107]:
df1 = get_figure(defenders[(defenders['sum_total_points']>0)], 
                 'sum_total_points', 
                 ['sum_clean_sheets','sum_minutes', 'sum_bonus'],
                 '',
                 map_position_to_color('Defender'))
plotly.offline.iplot(df1)

## Hero or zero?

In [110]:
errors_def = defenders['sum_errors_leading_to_goal'] 
+ defenders['sum_errors_leading_to_goal_attempt']
+ defenders['sum_penalties_conceded']
+ defenders['sum_own_goals']
+ defenders['sum_fouls']
+ defenders['sum_red_cards']
+ defenders['sum_yellow_cards']
+ defenders['sum_big_chances_missed']
+ defenders['sum_penalties_missed']
tmp_def = defenders.assign(errors=errors_def)
def2 = get_figure(tmp_def[(tmp_def['sum_minutes']>0)], 
                 'errors', 
                 ['sum_recoveries','sum_tackles'],
                 '',
                 map_position_to_color('Defender'))
plotly.offline.iplot(def2)